In [1]:
!pip install torch
!pip install transformers==4.34.1
!pip install --upgrade mlflow
!pip install  sentence_transformers


[notice] A new release of pip available: 22.2.2 -> 23.3.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip available: 22.2.2 -> 23.3.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip available: 22.2.2 -> 23.3.1
[notice] To update, run: pip install --upgrade pip
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 5.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 94.8 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 308.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 351.6 MB/s eta 0:00:00
  Created wheel for sentence_transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125925 sha256=5ba7b912a3a4b092fe7950b4d0cf861ab1f02cfecb88e3ea59d3d4c43c8850b4
  Stored in directory: /tmp/pip-ephem-wheel-cache-jatxntwx/wheels/71/67/06/162a3760c40d74dd40bc855d527008d26341c2b0ecf3e

In [3]:
import torch
import os
import mlflow
from transformers import AutoModelForSequenceClassification, AutoTokenizer, BatchEncoding, PreTrainedTokenizerFast
from transformers.modeling_outputs import SequenceClassifierOutput
import json
import mlflow.pytorch
import numpy as np
import requests
import logging
from sentence_transformers import CrossEncoder

MLFLOW_ROUTE = os.getenv("MLFLOW_ROUTE")
mlflow.set_tracking_uri(MLFLOW_ROUTE)
mlflow.set_experiment("vectara")
#mlflow.pytorch.autolog(registered_model_name="ranker")
logging.getLogger("mlflow").setLevel(logging.DEBUG)
tokenizer = AutoTokenizer.from_pretrained('vectara/hallucination_evaluation_model')
model = AutoModelForSequenceClassification.from_pretrained('vectara/hallucination_evaluation_model')
#model.eval()
mlflow.end_run()
with mlflow.start_run():
    components = {
        "model": model,
        "tokenizer": tokenizer,
    }
    mlflow.transformers.log_model(components, "vectara")

2023/11/20 14:03:52 INFO mlflow.tracking.fluent: Experiment with name 'vectara' does not exist. Creating a new experiment.


In [ ]:
import mlflow
from transformers.modeling_outputs import SequenceClassifierOutput
logged_model = 'runs:/384eb4f26ddc4bb08cea3987017c5b8d/rankerhao'

# Load model as a PyFuncModel.
loaded_model = mlflow.pyfunc.load_model(logged_model)

# Predict on a Pandas DataFrame.
import pandas as pd
batch_dict = encode(tokenizer, 'Is RedHat OpenShift Good', 'RedHat OpenShift is good')
data = pd.DataFrame(batch_dict["input_ids"])
#data.astype(float)
outputs: SequenceClassifierOutput = loaded_model.predict(batch_dict["input_ids"].numpy())